数据来源平台：爬取印尼四大电商平台数据：lazada，shopee，tokopedia，bukalapak

爬取品类 ：按顺序依次为：鞋，表，包，饰品

价位区间：按200-300，及300以上两个价位区间爬取(饰品稍等，待观察各品台基础价位)

销量：爬出月销量1500以上的sku

https://www.tokopedia.com/
https://www.bukalapak.com/

## shopee
月销
## 奇怪 Get 不到 product name

In [7]:
from selenium import webdriver
import time,re,csv
from pyquery import PyQuery as pq
import requests
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.expected_conditions as EC
import selenium.webdriver.support.ui as WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.keys import Keys

def get_list(class_name,class_url,save_file,monthly_sales):
    
    # Chrome object

    print('Getting class item list from Shopee ...')
    driver = webdriver.Chrome()  
    driver.maximize_window()
    driver.get(class_url) 
    time.sleep(2)

    for i in range(4):
        driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        

    items = driver.find_elements_by_css_selector('div.col-xs-2-4.shopee-search-item-result__item')
    item_number = len(items)
    if item_number == 50:
        with open(save_file,'a',encoding='utf-8') as w:    
            for i in items:
                #print(i.find_element_by_css_selector('._1NoI8_._2gr36I'))
                
                #pro_name = i.find_element_by_css_selector('div._1NoI8_._2gr36I').text
                pro_name = i.find_element_by_css_selector("[class^=_1NoI8_]").text
                price_info  = i.find_element_by_css_selector('div._36zw98').text
                
                price_range = re.sub(r'\.','',price_info)
    
                if '-' in price_range:
                    a = re.findall('Rp(.*) - Rp(.*)',price_range)
                    price = (int(a[0][0])+int(a[0][1]))/2
                else:
                    price = int(re.search('Rp(.*)',price_range).group(1))
                
                info = i.find_element_by_css_selector('div._2-i6yP').text
                monthly_sales = int(re.search('(\d+) Terjual',info).group(1))
                pro_url = i.find_element_by_tag_name('a').get_attribute('href')
                pic_info = i.find_element_by_css_selector('div._1T9dHf._3XaILN').get_attribute('style')
                pic_url = re.search(r'url\("(.*)"\);',pic_info).group(1)
                if monthly_sales >= sales_select:
                    print('class_name:{}\nproduct name:{}\nprice_info:{}\nprice:{}\nproduct url:{}\nmonthly sales:{}\npicture url:{}\n'.format(class_name,pro_name,price_info,price,pro_url,monthly_sales,pic_url))
                    #w.write('{},{}\n'.foramt(url,price))
                
    else:
        print('number of items is{}, waiting 10 seconds to reload'.foramt(item_number))
        #time.sleep(10)
        #get_list(class_url,save_file)

    
class_name = '靴子'
class_url = 'https://shopee.co.id/Boots-cat.35.1033?page=0&sortBy=sales'
save_file = r'class_item_list_over3000.csv'
sales_select = 1500

get_list(class_name,class_url,save_file,sales_select)

Getting class item list from Shopee ...
class_name:靴子
product name:
price_info:Rp198.000 - Rp279.000
price:238500.0
product url:https://shopee.co.id/Sepatu-Boots-Humm3r-Golem-size-39-44-dan-45-48-i.40377301.933066496
monthly sales:1577
picture url:https://cf.shopee.co.id/file/c11757c079d3611d2ff9e2beef2ef41a_tn



## pyquery

### 奇怪 get 不到 product url
`pro_url = i.find('div a').attr('href') `

#### 原因是打印出html代码，会发现包含  `xmlns="http://www.w3.org/1999/xhtml"`

```
<div xmlns="http://www.w3.org/1999/xhtml" class="col-xs-2-4 shopee-search-item-result__item"><div><a href="/Sepatu-Boots-Humm3r-Golem-size-39-44-dan-45-48-i.40377301.933066496"><div class="_1gkBDw"><div class="_3ZDC1p"><div class="_1T9dHf _3XaILN" style="background-image: url(&quot;https://cf.shopee.co.id/file/c11757c079d3611d2ff9e2beef2ef41a_tn&quot;)
```

但是pyquery中的选择器并没有错误，但是运行结果一直是None。这是为什么呢？后来通过查看相关文档得知，pyquery解析的是html类型的字符串，但是上面的类型是xhtml，所以会获取不到元素。可以在pq()方法初始化字符串时加上parser="html"告诉pyquery使用html规范解析，即可解决上述问题。
https://blog.csdn.net/qq_40176258/article/details/85041089

`pq(html,parser="html")`


再次打印出html代码，会发现已经不包含  `xmlns="http://www.w3.org/1999/xhtml"`

[官方文档](https://pythonhosted.org/pyquery/tips.html?highlight=xhtml)

In [ ]:
from selenium import webdriver
import time,re,csv,pymongo,os
from pyquery import PyQuery as pq
import requests
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.keys import Keys


def get_list(driver,save_file,category,class_name,class_url,url_format,sales_select,page):
    
    print('page {} ...'.format(page)) 
    driver.get(class_url.format(url_format,page)) 
    time.sleep(2)
    for i in range(5):
        driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
        time.sleep(4)
    pic_num = len(driver.find_elements_by_css_selector('._1w9jLI._37ge-4._2XtIUk'))
    print('picture load {}/50...'.format(pic_num))
    while (pic_num<49):
        driver.refresh()
        for i in range(4):
            driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
        for i in range(4):
            driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_UP)
            time.sleep(2)
        pic_num = len(driver.find_elements_by_css_selector('._1w9jLI._37ge-4._2XtIUk'))

    html = driver.page_source
    doc = pq(html,parser="html")
    item_list = doc('.col-xs-2-4.shopee-search-item-result__item')
    num = 0
    with open(save_file,'a',encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        for i in item_list.items():
            try:
                info = i.find('._2-i6yP').text()
                print(info)
                monthly_sales_info = re.search('(\d+) Terjual',info)
                if monthly_sales_info:
                    monthly_sales = int(monthly_sales_info.group(1))
                if monthly_sales >= sales_select:
                    num += 1
                    pro_name = i.find('._1NoI8_._2gr36I').text()
                    pro_name = re.sub(r',',' ',pro_name)
                    price_info  = i.find('._1w9jLI._37ge-4._2XtIUk').text()
                    price_range = re.sub(r'\.|\n','',price_info)
                    if '-' in price_range:
                        a = re.findall('Rp(.*) - Rp(.*)',price_range)
                        price = (int(a[0][0])+int(a[0][1]))/2
                    else:
                        price = int(re.search('Rp(.*)',price_range).group(1))
                    prefix = 'https://shopee.co.id'
                    pro_url = prefix+i.find('div a').attr('href') 
                    pic_info = i.find('._1T9dHf._3XaILN').attr('style')
                    print(type(pic_info))
                    pic_url = re.search(r'url\("(.*)_tn"\);',str(pic_info)).group(1).strip()
                    print('category:{}\nclass name:{}\nproduct name:{}\nprice info:{}\nprice:{}\nproduct url: {}\nmonthly sales:{}\npicture url: {}\n'.format(category,class_name,pro_name,price_range,price,pro_url,monthly_sales,pic_url))
                    csvfile.write('{},{},{},{},{},{},{},{}\n'.format(category,class_name,pro_name,price_range,price,monthly_sales,pro_url,pic_url)) 
                    pic_name = ' '.join(re.sub('[^a-z0-9A-Z_\-\s]','',pro_name,count=0).split())
                    save_image(pic_url,pic_name,category)

                    item = {
                        'category':category,
                        'main class':class_name,
                        'item name':pro_name,
                        'price info':price_range,
                        'price':price,
                        'monthly sales':monthly_sales,
                        'item url':pro_url,
                        'item picture':pic_url
                    }
                    save_to_mongo(item)
            except:
                print('something got worng, pass this item {}'.format(pro_name))
                continue
    print(num,len(item_list))
    if num == len(item_list):
        page += 1
        get_list(driver,save_file,category,class_name,class_url,url_format,sales_select,page)
                
        
def save_image(img_url,pic_name,category):
    print('{} saving picture... {}'.format(pic_name,img_url))
    img_response = requests.get(img_url,headers=headers)
    path = r'{}'.format(category)
    isExists = os.path.exists(path)
    if not isExists:
        os.makedirs(path)
    with open(r'{}\{}.png'.format(path,pic_name),'ab') as f:
        f.write(img_response.content)          
                        
def save_to_mongo(result):
    MONGO_URL = 'localhost'
    MONGO_DB = 'sales_crawl'
    MONGO_COLLECTION = 'shopee_sales_1500_0305'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')


def get_url(file,driver,save_file,sales_select,page): 
    
    with open(save_file,'a',encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['category','main class','item name','price info','price','monthly sales','item url','item picture']) 
    with open(file,'r',encoding='utf-8') as f:
        reader = csv.reader(f)
        rows = [row for row in reader][-3:]
        for row in rows:
            category = row[0]
            class_name = row[1]
            url_format = row[2]
            get_list(driver,save_file,category,class_name,class_url,url_format,sales_select,page)
            
file = 'sales.csv'  
headers = {'User-Agent': 'User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}            
#category = 'MenShoes'
#class_name = 'Boots'
#class_url = 'https://shopee.co.id/Boots-cat.35.1033?page={}&sortBy=sales'

class_url = 'https://shopee.co.id/{}?page={}&sortBy=sales'
save_file = r'shopee_sales_1500_0305.csv'
sales_select = 1500
page = 0

if __name__ == '__main__':
    driver = webdriver.Chrome()
    driver.maximize_window()
    get_url(file,driver,save_file,sales_select,page)
    driver.close()

In [6]:
import csv

def get_url(file):
    
    with open(file,'r',encoding='utf-8') as f:
        reader = csv.reader(f)
        rows = [row for row in reader][1:]
        for row in rows:
            print(row[0],row[1],row[2])
            
file = 'sales.csv'
get_url(file)

MenShoes Boots Boots-cat.35.1033
MenShoes CasualShoes Sepatu-Kasual-cat.35.1027
MenShoes Sandals Sandal-cat.35.1019
MenShoes Sneakers Sneakers-cat.35.1013
MenShoes Formal Sepatu-Formal-cat.35.1031
Watches MenWatches Jam-Tangan-Pria-cat.37.12452
Watches WomenWatches Jam-Tangan-Wanita-cat.37.12443
Watches CoupleWatches Jam-Tangan-Couple-cat.37.12461
Menbags Backpacks Tas-Punggung-cat.10217.10222
Menbags ToteBags Tas-Tote-cat.10217.12621
Menbags MessengeBags Tas-Messenger-cat.10217.18222
Menbags SlingBags Tas-Selempang-cat.10217.18224
Menbags WaistBags Tas-Pinggang-cat.10217.12631
Menbags LaptopBags Tas-Laptop-cat.10217.10223
Menbags Shoulder Bags Tas-Bahu-cat.10217.10224
Menbags ClutchBags Clutch-cat.10217.18227
Menbags SportsBags Tas-Olahraga-cat.10217.12626
Menbags MenWallet Dompet-Pria-cat.10217.10219
Fashion HairAccessories Aksesoris-Rambut-cat.38.1145
Fashion Hats Topi-cat.38.1147
Fashion Glasses Kacamata-cat.38.1143
Fashion ContactLenses Lensa-Kontak-cat.38.14983
Fashion Earrings A

### 报错 TypeError: expected string or bytes-like object
（预定的数据类型或者字节对象相关） 一般为数据类型不匹配造成的

```
---> 49                 pic_url = re.search(r'url\("(.*)_tn"\);',pic_info).group(1).strip()
     50                 print('category:{}\nclass name:{}\nproduct name:{}\nprice info:{}\nprice:{}\nproduct url: {}\nmonthly sales:{}\npicture url: {}\n'.format(category,class_name,pro_name,price_range,price,pro_url,monthly_sales,pic_url))
     51                 csvfile.write('{},{},{},{},{},{},{},{}\n'.format(category,class_name,pro_name,price_range,price,monthly_sales,pro_url,pic_url))

c:\users\win7\appdata\local\programs\python\python36-32\lib\re.py in search(pattern, string, flags)
    180     """Scan through string looking for a match to the pattern, returning
    181     a match object, or None if no match was found."""
--> 182     return _compile(pattern, flags).search(string)
    183 
    184 def sub(pattern, repl, string, count=0, flags=0):

TypeError: expected string or bytes-like object
```
### 原因是 
`pic_info = i.find('._1T9dHf._3XaILN').attr('style')` 返回的是 `nonetype` 类型


用正则表达式处理数据时时出现错误,正则是用来匹配字符串类型，
因此在正则表达式之前，转换一下数据类型，就解决了问题
str()

[参考文档](https://blog.csdn.net/weixin_42105977/article/details/80390957)

shopee.py

In [ ]:
# -*- coding: utf-8 -*-
from scrapy import Request, Spider
from urllib.parse import urlencode
import re
from sales.items import ProductItem


class ShopeeSpider(Spider):
	name = 'shopee'
	allowed_domains = ['shopee.co.id']
	base_url = 'https://shopee.co.id/Boots-cat.35.1033?'
	sales_select = 800

	def start_requests(self):
		data = {'sortBy':'sales'}
		for page in range(0,2):
			data['page'] = page
			params =urlencode(data)
			url = self.base_url + params
			yield Request(url, self.parse)

	def parse(self,response):
		item_list = response.css('.col-xs-2-4.shopee-search-item-result__item')
		for i in item_list:
			item = ProductItem()
			item['category'] = 'MenShoes'
			item['class_name'] = 'Boots'
			info = i.css('._2-i6yP::text').extract_first()
			item['monthly_sales'] = int(re.search('(\d+) Terjual',info).group(1)) 
			if monthly_sales >= self.sales_select:
				item['pro_name'] = i.css('._1NoI8_._2gr36I::text').extract_first()
				price_info  = i.css('._1w9jLI._37ge-4._2XtIUk::text').extract_first()
				item['price_range'] = re.sub(r'\.|\n','',price_info)
				if '-' in price_range:
					a = re.findall('Rp(.*) - Rp(.*)',price_range)
					item['price'] = (int(a[0][0])+int(a[0][1]))/2
				else:
					item['price'] = int(re.search('Rp(.*)',price_range).group(1))
				prefix = 'https://shopee.co.id'
				item['pro_url'] = prefix+i.css('div a::attr("href")').extract_first()
				pic_info = i.css('._1T9dHf._3XaILN::attr("style")').extract_first()
				item['pic_url'] = re.search(r'url\("(.*)_tn"\);',str(pic_info)).group(1).strip()
				item['pic_name'] = ' '.join(re.sub('[^a-z0-9A-Z_\-\s]','',pro_name,count=0).split())
				yield item


middlewares.py

In [ ]:
# -*- coding: utf-8 -*-

# Define here the models for your spider middleware
#
# See documentation in:
# https://doc.scrapy.org/en/latest/topics/spider-middleware.html

#from scrapy import signals
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.ui as WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.keys import Keys

from scrapy.http import HtmlResponse
from logging import getLogger
import time


class SalesDownloaderMiddleware(object):
    # Not all methods need to be defined. If a method is not defined,
    # scrapy acts as if the downloader middleware does not modify the
    # passed objects.

    def __init__(self, timeout=None, service_args=[]):
        self.logger = getLogger(__name__)
        self.timeout = timeout
        self.driver = webdriver.PhantomJS(service_args=service_args)
        self.driver.set_window_size(1400,700)
        self.driver.set_page_load_timeout(self.timeout)
        self.wait = WebDriverWait(self.driver, self.timeout)

    def __del__(self):
        self.driver.close()

    def process_request(self, request, spider):
        # Called for each request that goes through the downloader
        # middleware.

        # Must either:
        # - return None: continue processing this request
        # - or return a Response object
        # - or return a Request object
        # - or raise IgnoreRequest: process_exception() methods of
        #   installed downloader middleware will be called
        self.logger.debug('PhantomJS is starting')
        try:
            self.driver.get(request.url)
            for i in range(4):
                    self.driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
                    time.sleep(2)
            self.wait.until(EC.presence_of_element_located(BY.CSS_SELECTOR, '.col-xs-2-4.shopee-search-item-result__item'))
            return HtmlResponse(url=request.url, body=self.driver.page_source,request=request,encoding='utf-8', status=200)

        except TimeoutException:
            return HtmlResponse(url=request.url, status=500, request=request)

    @classmethod
    def from_crawler(cls, crawler):
        # This method is used by Scrapy to create your spiders.
        return cls(timeout=crawler.settings.get('SELENIUM_TIMEOUT'),
            service_args=crawler.settings.get('PHANTOMJS_SERVICE_ARGS'))




    def process_response(self, request, response, spider):
        # Called with the response returned from the downloader.

        # Must either;
        # - return a Response object
        # - return a Request object
        # - or raise IgnoreRequest
        return response

    def process_exception(self, request, exception, spider):
        # Called when a download handler or a process_request()
        # (from other downloader middleware) raises an exception.

        # Must either:
        # - return None: continue processing this exception
        # - return a Response object: stops process_exception() chain
        # - return a Request object: stops process_exception() chain
        pass

    def spider_opened(self, spider):
        spider.logger.info('Spider opened: %s' % spider.name)



class SalesSpiderMiddleware(object):
    # Not all methods need to be defined. If a method is not defined,
    # scrapy acts as if the spider middleware does not modify the
    # passed objects.

    @classmethod
    def from_crawler(cls, crawler):
        # This method is used by Scrapy to create your spiders.
        s = cls()
        crawler.signals.connect(s.spider_opened, signal=signals.spider_opened)
        return s

    def process_spider_input(self, response, spider):
        # Called for each response that goes through the spider
        # middleware and into the spider.

        # Should return None or raise an exception.
        return None

    def process_spider_output(self, response, result, spider):
        # Called with the results returned from the Spider, after
        # it has processed the response.

        # Must return an iterable of Request, dict or Item objects.
        for i in result:
            yield i

    def process_spider_exception(self, response, exception, spider):
        # Called when a spider or process_spider_input() method
        # (from other spider middleware) raises an exception.

        # Should return either None or an iterable of Response, dict
        # or Item objects.
        pass

    def process_start_requests(self, start_requests, spider):
        # Called with the start requests of the spider, and works
        # similarly to the process_spider_output() method, except
        # that it doesn’t have a response associated.

        # Must return only requests (not items).
        for r in start_requests:
            yield r

    def spider_opened(self, spider):
        spider.logger.info('Spider opened: %s' % spider.name)




pipeline

In [ ]:
# -*- coding: utf-8 -*-

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://doc.scrapy.org/en/latest/topics/item-pipeline.html

import pymongo

class MongoPipeline(object):
	def __init__(self,mongo_uri,mongo_db):
		self.mongo_uri = mongo_uri
		self.mongo_db = mongo_db

	@classmethod
	def from_crawler(cls, crawler):
		return cls(mongo_uri=crawler.settings.get('MONGO_URI'),mongo_db=crawler.settings.get('MONGO_DB'))

	def open_spider(self,spider):
		self.client = pymongo.MongoClient(self.mongo_uri)
		self.db = self.client[self.mongo_db]

	def process_item(self, item, spider):
		self.db['item.collection'].inset(dict(item))
		return item

	def close_spider(self,spider):
		self.client.close()


setting

In [ ]:
# -*- coding: utf-8 -*-

# Scrapy settings for sales project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://doc.scrapy.org/en/latest/topics/settings.html
#     https://doc.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://doc.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'sales'

SPIDER_MODULES = ['sales.spiders']
NEWSPIDER_MODULE = 'sales.spiders'



# Crawl responsibly by identifying yourself (and your website) on the user-agent
#USER_AGENT = 'sales (+http://www.yourdomain.com)'
#USER_AGENT = 'User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'


# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32

# Configure a delay for requests for the same website (default: 0)
# See https://doc.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and docs
#DOWNLOAD_DELAY = 3
# The download delay setting will honor only one of:
#CONCURRENT_REQUESTS_PER_DOMAIN = 16
#CONCURRENT_REQUESTS_PER_IP = 16

# Disable cookies (enabled by default)
#COOKIES_ENABLED = False

# Disable Telnet Console (enabled by default)
#TELNETCONSOLE_ENABLED = False

# Override the default request headers:
#DEFAULT_REQUEST_HEADERS = {
#   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#   'Accept-Language': 'en',
#}

# Enable or disable spider middlewares
# See https://doc.scrapy.org/en/latest/topics/spider-middleware.html
#SPIDER_MIDDLEWARES = {
#    'sales.middlewares.SalesSpiderMiddleware': 543,
#}

# Enable or disable downloader middlewares
# See https://doc.scrapy.org/en/latest/topics/downloader-middleware.html
DOWNLOADER_MIDDLEWARES = {
    'sales.middlewares.SalesDownloaderMiddleware': 543,
}
SELENIUM_TIMEOUT = 60

# Enable or disable extensions
# See https://doc.scrapy.org/en/latest/topics/extensions.html
#EXTENSIONS = {
#    'scrapy.extensions.telnet.TelnetConsole': None,
#}

# Configure item pipelines
# See https://doc.scrapy.org/en/latest/topics/item-pipeline.html
ITEM_PIPELINES = {
	'sales.pipelines.MongoPipeline': 300,
}
MONGO_URI = 'localhost'
MONGO_DB = 'shopee'

# Enable and configure the AutoThrottle extension (disabled by default)
# See https://doc.scrapy.org/en/latest/topics/autothrottle.html
#AUTOTHROTTLE_ENABLED = True
# The initial download delay
#AUTOTHROTTLE_START_DELAY = 5
# The maximum download delay to be set in case of high latencies
#AUTOTHROTTLE_MAX_DELAY = 60
# The average number of requests Scrapy should be sending in parallel to
# each remote server
#AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0
# Enable showing throttling stats for every response received:
#AUTOTHROTTLE_DEBUG = False

# Enable and configure HTTP caching (disabled by default)
# See https://doc.scrapy.org/en/latest/topics/downloader-middleware.html#httpcache-middleware-settings
#HTTPCACHE_ENABLED = True
#HTTPCACHE_EXPIRATION_SECS = 0
#HTTPCACHE_DIR = 'httpcache'
#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'


item

In [ ]:
# -*- coding: utf-8 -*-

# Define here the models for your scraped items
#
# See documentation in:
# https://doc.scrapy.org/en/latest/topics/items.html

from scrapy import Item, Field


class ProductItem(Item):

	collection = 'products'
	category = Field()
	class_name = Field()
	pro_name = Field()
	price_range = Field()
	price = Field()
	monthly_sales = Field()
	pro_url = Field()
	pic_url  = Field()
	pic_name = Field()
